In [1]:
import fitz
import re
import os 
import pandas as pd
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

# Define Function 

In [22]:
def hope_bank_to_csv(pdf_path: str,
                     excel_path='C:/Users/trilnd/Desktop/QBO/Test_file/'):
       #Function to get description

    def get_description(trans,len_upper,len_lower):
        if re.search('bankofthewest.com',all_text_page):
            return trans[len_upper+len_lower+1:]
        else:
        # check space: 
            check_space = pat+'(?=\s)'
            if re.search(check_space,trans) != None:
                return trans[len_upper+1:(len(trans)-len_lower)]
            else:
                return trans[(len_upper):(len(trans)-len_lower)]

    pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
    pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{4})?)'
    num_pat = r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'
    base_name = os.path.basename(pdf_path)[0:-4]
    pdf_dict = {base_name:{'Status':'',
                           'Transaction':'',
                           'Dataframe':''}}
    
    #1: Check whether the path is correct or wrong
    try:
        with open(pdf_path,'rb') as file:
            print(f"{base_name}: Connect file successful")
            pdf_dict[base_name]["Status"] = "Connect file successful"
            
    except IOError:
            print("Can not find the file directory or file name does not exist")
            pdf_dict[base_name]["Status"] = "Can not find the file directory or file name does not exist"
            return pdf_dict

    #2: Implement to convert file to pdf variable and print total number pages
    else:
        try:
            pdf_file = fitz.open(pdf_path)
        except Exception as e:
            raise e
        else:
            
#################################### GET TRANSACTION ########################################### 

        #3: Convert pdf file to the string 
            all_text_page =''
            for page in pdf_file:
                text = page.get_text('text')
                all_text_page+= (text+'\n')

            if re.search(r'.',all_text_page) == None:
                raise Exception("This file without text data")
            else:
                all_text_list = [line.strip() for line in all_text_page.split('\n')]
            
        # : Check if the the file contain text or numeric?
            if re.search(r'.',all_text_page) is None:
                print("This PDF file is Scan data")
                pdf_dict[base_name]["Status"] = "This PDF file is Scan data"
                return pdf_dict
                     
        #4 Select proper date pattern for PDF file
            count_p1 = sum([1 for line in all_text_list if re.search(pattern_1, line)])
            count_p2 = sum([1 for line in all_text_list if re.search(pattern_2, line)])
            
            if count_p1 > count_p2:
                kind_extract_pattern = 1
                pat = pattern_1
                #print("Chose the patern 1")
                  
            elif  count_p1 < count_p2:
                kind_extract_pattern = 2
                pat = pattern_2
                #print("Chose the patern 2")
            else:
                print("ERROR: There is not proper pattern for this file\n\n")
                pdf_dict[base_name]["Status"] = "ERROR: There is not proper pattern for this file"
                return pdf_dict

        #5: Extract Transaction from PDF
            extract_list = []
            for i in range(len(all_text_list)):
                line = all_text_list[i]

                if re.search(pat,line) and re.search(num_pat,line):
                    extract_list.append(line.strip())

                elif re.search(pat,line):
                    extract_list.append(line+' '+all_text_list[i+1])

                elif re.search(num_pat,line):
                    span = re.search(num_pat,line).span()
                    only_numeric = line[span[0]:span[1]]
                    extract_list.append(only_numeric)

        #6: GET ALL TRANSACTION START WITH DATE OR COMBINE ROW (START WITH DATE) AND ROW NUMBER
            filter_list = []
            for i in range(len(extract_list)):
                if len(extract_list)-1 > i:
                    line = extract_list[i]
                    # if this line contains both of date and amount => append to filter list
                    if re.search(pat,line) and re.search(num_pat,line):
                        span = re.search(num_pat,line).span()
                        line = line[0:span[1]]
                        filter_list.append(line.strip())

                    # if this line is date = > combine with next line and append to next list
                    elif re.search(pat,line):
                        next_line = extract_list[i+1]

                        # if next line is date get next second line
                        if re.search(pat,next_line) != None:
                            try:
                                new_line = line +' '+ extract_list[i+2]
                                filter_list.append(new_line)
                            except IndexError:
                                pass
                        else:
                            new_line = line +' '+ next_line
                            filter_list.append(new_line)

                    # if this line is number: => continue 
                    elif re.search(num_pat,line):
                        continue
                        
#################################### GET CHECK NUMBER ###########################################

        # Pattern define:
            pat_check_no = r'\b\d+\*?\b'
            pat_date =  r'((\d{1,2}/\d{1,2})(/\d{4})?)'
            pat_num = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
            
         #1: Get Page and Coodinator contain "Checks" word
            String = 'Checks'
            Pagelist = []
            for i in range(0, number_page):
                PageObj = pdf_file[i]
                Text = PageObj.get_text('text')
                ReSearch = re.search(String, Text, flags=re.IGNORECASE)
                if ReSearch != None:
                    # print(re.findall(String,Text))
                    info = (i, ReSearch.span()[0])
                    Pagelist.append(info)
                    
        #2: Convert string to list
            checkno_text_page = ''
            for i in Pagelist:
                checkno_text_page += (pdf_file[i[0]].get_text('text')[i[1]:] + '\n')
                
        # 3: Get line without Alphabelt digit
            filter_checkno_list = [line.strip() for line in checkno_text_page.split('\n') if re.search(r'[a-zA-Z+]',line) is None]
        
        # 4: Get only checkno line
            check_no_list = []
            for i in range(len(filter_checkno_list)):
                if i <len(filter_checkno_list)-1:
                    line = filter_checkno_list[i]
                    next_line = filter_checkno_list[i+1]
                    if re.search(pat_check_no,line) and re.search(pat_date,line):
                        if re.search(pat_num,next_line):
                            check_no_line = line+' '+next_line
                            check_no_list.append(check_no_line.split())
            

################################### CONVERT TO DATAFRAME AND EXPORT TO FILE CSV ###############################################

            #################### TRANSACTION DATAFRAME #########################
            trans_df = pd.DataFrame(filter_list,columns=['Transaction'])

            # Get only transaction which contaims amount
            trans_df = trans_df[~trans_df.Transaction.apply(lambda x: re.search(num_pat,x)).isnull()].reset_index(drop=True)

            # Print total transaction in file pdf
            print(f"File {base_name}.pdf: {trans_df.shape[0]} transactions\n\n")
            pdf_dict[base_name]["Transaction"] = f'File {base_name}.pdf: {trans_df.shape[0]} transactions'
        
            # Create 2 columns Post date and Trans date
            if kind_extract_pattern == 2:
    
                # Define pattern
                kind_2_pat_2d = r'(\d{1,2}[/-]\d{1,2})\s?(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_1d = r'(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_has_year = r'(\d{1,2}[/-]\d{1,2}[/-]\d{4})'

                # Create filter 
                has_year = trans_df.Transaction.str.contains(pat=kind_2_pat_has_year).sum()/trans_df.shape[0]
                has_2date = trans_df.Transaction.str.contains(pat=kind_2_pat_2d).sum()/trans_df.shape[0]

                # Check case 1: Has year ?
                if has_year > 0.7:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_has_year)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0]

                # Check case 2: Has two date? Post date and Trans date
                elif has_2date > 0.7:
                    trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_2_pat_2d)

                    # If there are some transaction having 1 date => get this date to Post day
                    outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
                    trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_2_pat_1d)

                else:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0] 

            # Fill NA
            trans_df = trans_df.fillna('')
     
            # Create Amount columns     
            trans_df['Amount'] = trans_df.Transaction.str.extract(pat=num_pat,expand=False).str.replace(pat=r'[$\s]',repl='',regex=True)

            # Create the Description columns
            trans_df['len_upper']=trans_df[['Trans_date','Post_date']].apply(lambda x: len(x['Trans_date'])+len(x['Post_date']), axis=1)
            trans_df['len_lower'] = trans_df.Transaction.str.extract(num_pat,expand=False).apply(lambda x: len(x))

            trans_df['Description'] = trans_df.apply(lambda x: get_description(trans=x['Transaction'],
                                                                               len_upper=x['len_upper'],
                                                                               len_lower=x['len_lower']),axis=1).str.strip()
            
            # Replace the multiple space to one mutiple space 
            trans_df['Description'] = trans_df.Description.str.replace(regex=True,pat=r'\s+',repl=' ')
            columns = ['Transaction', 'Trans_date', 'Post_date', 'Amount','Description']
            
            # Convert to the excel file
            excel_file_path = os.path.join(excel_path, f'{base_name}.xlsx')
            trans_df[columns].to_excel(excel_writer=excel_file_path,index=False)  
            pdf_dict[base_name]["Dataframe"] = trans_df[columns]
            
            ####################### CHECK NO DATAFRAME #####################################
            check_no_df = pd.DataFrame(check_no_list,columns=['Check_no','Date','Amount'])
            check_no_df['Amount'] = check_no_df['Amount'].str.replace('$','',regex=False)
            # 6: Update file excel created before with new sheet "Check No"
            
            # if check path is valid
            if os.path.exists(excel_file_path):
                with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    check_no_df.to_excel(writer, sheet_name='Check_no', index=False)
            else:
                check_no_df.to_excel(path, sheet_name='Check_no', index=False)
                
            pdf_dict[base_name]["DF_Check_No"] = check_no_df
            
            return pdf_dict

# Get Transaction

In [45]:
def get_description(trans,len_upper,len_lower):
        if re.search('bankofthewest.com',all_text_page):
            return trans[len_upper+len_lower+1:]
        else:
        # check space: 
            check_space = pat+'(?=\s)'
            if re.search(check_space,trans) != None:
                return trans[len_upper+1:(len(trans)-len_lower)]
            else:
                return trans[(len_upper):(len(trans)-len_lower)]

In [4]:
print(fitz.__doc__)


PyMuPDF 1.21.0: Python bindings for the MuPDF 1.21.0 library.
Version date: 2022-11-08 00:00:01.
Built for Python 3.9 on win32 (64-bit).



In [5]:
pdf_path =  r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Capital One CC 7682 (0036) 2022\2022.02.16.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Hanmi Checking 7666 2022\2022 11_30_2022-document.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of the West Checking 8318 2022\Statement 02_28_22.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of the West CC 5867 2022\2022.02.28.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of the West Checking 8318 2022\Statement 02_28_22.pdf'
#r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of Hope Checking 0933 2022\20221021-statements-4293-.pdf'
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.04.pdf' 

base_name = os.path.basename(pdf_path)[0:-4]
dict_info = {base_name:{}}
base_name

'2022.02.16'

In [6]:
pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2,4})?)'
pattern_3 = r'^((\d{1,2}-\d{1,2})(-\d{2,4})?)'
num_pat = r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'

#1: Check whether the path is correct or wrong
try:
    with open(pdf_path,'rb') as file:
        print("Connect file successful")
except IOError:
        print("Can not find the file directory or file name does not exist")

#2: Implement to convert file (Beautiful Transaction) to pdf variable and print total number pages
else:
    try:
        pdf_file = fitz.open(pdf_path)
    except Exception as e:
        raise e
    else:
        number_page = pdf_file.page_count
        print("Total number pages of File PDF are",number_page)

Connect file successful
Total number pages of File PDF are 6


In [6]:
#doc = fitz.open(path)

In [7]:
pdf_file.metadata

{'format': 'PDF 1.7',
 'title': 'Card_Statement',
 'author': 'Registered to: CAPITAL1',
 'subject': '',
 'keywords': '',
 'creator': 'OpenText Exstream Version 16.6.1 64-bit',
 'producer': '',
 'creationDate': '2/17/2022 09:29:14',
 'modDate': '',
 'trapped': '',
 'encryption': None}

In [26]:
all_text_page =''
for page in pdf_file:
    text = page.get_text('text')
    all_text_page+= (text+'\n')
    
if re.search(r'.',all_text_page) == None:
    raise Exception("This file without text data")
else:
    all_text_list = [line.strip() for line in all_text_page.split('\n')]

In [24]:
#all_text_list

In [27]:
if re.search(r'.',all_text_page) == None:
    raise Exception("This is the file with scan data")
else:
    print('Success')

Success


In [48]:
if re.search('www.Hanmi.com', all_text_page) or re.search('capitalone.com', all_text_page) :
    num_pat = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
    print(1)
else:
    num_pat = r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'
    print(2)

1


In [49]:
#4 Select date pattern
count_p1 = len(re.findall(pattern_1[1:],all_text_page))
count_p2 = len(re.findall(pattern_2[1:],all_text_page))
#count_p3 = len(re.findall(pattern_3[1:],all_text_page))
 
if count_p1 > count_p2:
    kind_extract_pattern = 1
    pat = pattern_1
    print("Chose the patern 1")

elif count_p2 > count_p1:
    kind_extract_pattern = 2
    pat = pattern_2
    print("Chose the patern 2")
    
# elif count_p3 > count_p1 and count_p3 > count_p2:
#     kind_extract_pattern = 2
#     pat = pattern_3
#     print("Chose the patern 3")  

else:
    print("ERROR: There is not proper pattern for this file")

Chose the patern 1


In [50]:
#5: Extract Transaction from PDF
extract_list = []
    
if re.search('capitalone.com', all_text_page):
    for i, line in enumerate(all_text_list):     
        if re.search(pat,line) and re.search(num_pat,line):
            extract_list.append(line.strip())

        elif re.search(pat,line) and re.search(num_pat,all_text_list[i+3]):
            trans = all_text_list[i:i+4]
            extract_list.append(' '.join(trans))

#         elif re.search(num_pat,line):
#             span = re.search(num_pat,line).span()
#             only_numeric = line[span[0]:span[1]]
#             extract_list.append(only_numeric)
            
else: 
    
    for i in range(len(all_text_list)):
        line = all_text_list[i]

        if re.search(pat,line) and re.search(num_pat,line):
            extract_list.append(line.strip())

        elif re.search(pat,line):
            extract_list.append(line+' '+all_text_list[i+1])

        elif re.search(num_pat,line):
            span = re.search(num_pat,line).span()
            only_numeric = line[span[0]:span[1]]
            extract_list.append(only_numeric)
        

In [51]:
extract_list

['Feb 4 Feb 4 CREDIT-CASH BACK REWARD - $456.56',
 'Feb 11 Feb 12 VFS Services USA Inc.101-23456789DC - $178.00',
 'Feb 12 Feb 14 WAL-MART #1879BARSTOWCA - $17.64',
 'Feb 13 Feb 14 CAPITAL ONE AUTOPAY PYMTAuthDate 13-Feb - $7,549.68',
 'Jan 18 Jan 20 RIVER ROAD X-PRESS CAR WACORONACA $14.00',
 'Jan 18 Jan 20 NEW SPICE WORLDCORONACA $9.05',
 'Jan 20 Jan 21 PEARSON DENTAL &MED SUPP818-362-2600CA $663.09',
 'Jan 20 Jan 22 NEW SPICE WORLDCORONACA $35.95',
 'Jan 22 Jan 24 INTUIT *QuickBooks OnlineCL.INTUIT.COMCA $25.00',
 'Jan 22 Jan 24 CAFE RIO 0110 RANCHO CUCRANCHO CUCAMOCA $11.43',
 'Jan 22 Jan 24 7-ELEVEN 33308RANCHO CUCAMOCA $99.76',
 'Jan 24 Jan 25 CROWN WORLD INC949-440-2784CA $1,399.23',
 'Jan 24 Jan 25 GLACIER WATER VENDING 2WINSTON SALEMNC $1.85',
 'Jan 24 Jan 25 GLACIER WATER VENDING 2WINSTON SALEMNC $1.85',
 'Jan 25 Jan 26 WAL-MART #2487CORONACA $289.04',
 'Jan 26 Jan 27 DD/BR #357492BARSTOWCA $6.04',
 'Jan 29 Jan 31 WAL-MART #1879BARSTOWCA $22.89',
 'Jan 29 Jan 31 CVWD WATER PM

In [52]:
# 6: GET ALL TRANSACTION START WITH DATE OR COMBINE ROW (START WITH DATE) AND ROW NUMBER
filter_list = []
if re.search('bankofthewest.com',all_text_page):
    for i, line in enumerate(extract_list):
        if re.search(pat,line) and re.search(num_pat,line):
            filter_list.append(line.strip())
            
elif re.search('capitalone.com', all_text_page):
    for i, line in enumerate(extract_list):
        if re.search(pat,line) and re.search(num_pat,line):
            filter_list.append(line.strip())
else:
    for i in range(len(extract_list)):
        if len(extract_list)-1 > i:
            line = extract_list[i]
            # if this line contains both of date and amount => append to filter list
            if re.search(pat,line) and re.search(num_pat,line):
                span = re.search(num_pat,line).span()
                line = line[0:span[1]]
                filter_list.append(line.strip())

            # if this line is date = > combine with next line and append to next list
            elif re.search(pat,line):
                next_line = extract_list[i+1]

                # if next line is date get next second line
                if re.search(pat,next_line) != None:
                    try:
                        new_line = line +' '+ extract_list[i+2]
                        filter_list.append(new_line)
                    except IndexError:
                        pass
                else:
                    new_line = line +' '+ next_line
                    filter_list.append(new_line)

            # if this line is number: => continue 
            elif re.search(num_pat,line):
                continue

In [53]:
filter_list

['Feb 4 Feb 4 CREDIT-CASH BACK REWARD - $456.56',
 'Feb 11 Feb 12 VFS Services USA Inc.101-23456789DC - $178.00',
 'Feb 12 Feb 14 WAL-MART #1879BARSTOWCA - $17.64',
 'Feb 13 Feb 14 CAPITAL ONE AUTOPAY PYMTAuthDate 13-Feb - $7,549.68',
 'Jan 18 Jan 20 RIVER ROAD X-PRESS CAR WACORONACA $14.00',
 'Jan 18 Jan 20 NEW SPICE WORLDCORONACA $9.05',
 'Jan 20 Jan 21 PEARSON DENTAL &MED SUPP818-362-2600CA $663.09',
 'Jan 20 Jan 22 NEW SPICE WORLDCORONACA $35.95',
 'Jan 22 Jan 24 INTUIT *QuickBooks OnlineCL.INTUIT.COMCA $25.00',
 'Jan 22 Jan 24 CAFE RIO 0110 RANCHO CUCRANCHO CUCAMOCA $11.43',
 'Jan 22 Jan 24 7-ELEVEN 33308RANCHO CUCAMOCA $99.76',
 'Jan 24 Jan 25 CROWN WORLD INC949-440-2784CA $1,399.23',
 'Jan 24 Jan 25 GLACIER WATER VENDING 2WINSTON SALEMNC $1.85',
 'Jan 24 Jan 25 GLACIER WATER VENDING 2WINSTON SALEMNC $1.85',
 'Jan 25 Jan 26 WAL-MART #2487CORONACA $289.04',
 'Jan 26 Jan 27 DD/BR #357492BARSTOWCA $6.04',
 'Jan 29 Jan 31 WAL-MART #1879BARSTOWCA $22.89',
 'Jan 29 Jan 31 CVWD WATER PM

***#6: Convert list to dataFrame***

In [54]:
trans_df = pd.DataFrame(filter_list,columns=['Transaction'])

# Get only transaction which contaims amount
trans_df = trans_df[~trans_df.Transaction.apply(lambda x: re.search(num_pat,x)).isnull()].reset_index(drop=True)

# Print total transaction in file pdf
status= f"Total transaction in file PDF: {trans_df.size}"
dict_info[base_name]["Status"] = status
print(status)

Total transaction in file PDF: 67


In [55]:
dict_info

{'2022.02.16': {'Status': 'Total transaction in file PDF: 67'}}

In [56]:
# Create 2 columns Post date and Trans date

if kind_extract_pattern == 2:
    
    # Define pattern
    kind_1_pat_2d = r'(\d{1,2}[/-]\d{1,2})\s?(\d{1,2}[/-]\d{1,2})'
    kind_1_pat_1d = r'(\d{1,2}[/-]\d{1,2})'
    kind_1_pat_has_year = r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    
    # Create filter 
    has_year = trans_df.Transaction.str.contains(pat=kind_1_pat_has_year).sum()/trans_df.shape[0]
    has_2date = trans_df.Transaction.str.contains(pat=kind_1_pat_2d).sum()/trans_df.shape[0]
    
    # Check case 1: Has year ?
    if has_year > 0.7:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_has_year)
        trans_df['Trans_date'] = ['']*trans_df.shape[0]
    
    # Check case 2: Has two date? Post date and Trans date
    elif has_2date > 0.7:
        trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_1_pat_2d)
        
        # If there are some transaction having 1 date => get this date to Post day
        outlier = (trans_df['Transaction'].str.contains(pat=kind_1_pat_2d) == False)
        trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_1_pat_1d)
        
    else:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_1d)
        trans_df['Trans_date'] = ['']*trans_df.shape[0] 

if kind_extract_pattern == 1:
    # Define pattern
    kind_2_pat_2d = r'([A-Z][a-z]{2}\s?\d{1,2})\s?([A-Z][a-z]{2}\s?\d{1,2})'
    kind_2_pat_1d = r'([A-Z][a-z]{2}\s?\d{1,2})'
    kind_2_pat_has_year = r'([A-Z][a-z]{2}\s?\d{1,2},?\s?\d{2,4})'
    
    # Create filter 
    has_2date = trans_df.Transaction.str.contains(pat=kind_2_pat_2d).sum()/trans_df.shape[0]
    has_year = trans_df.Transaction.str.contains(pat=kind_2_pat_has_year).sum()/trans_df.shape[0]
    
    # Check case 1: Has year ?
    if has_year > 0.7:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_has_year)
        trans_df['Trans_date'] = ['']*trans_df.shape[0]
    
    # Check case: Has two date? Post date and Trans date
    elif has_2date > 0.7:
        trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_2_pat_2d)
        
        # If there are some transaction having 1 date => get this date to Post day
        outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
        trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_2_pat_1d)
        
    else:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
        trans_df['Trans_date'] = ['']*trans_df.shape[0] 


In [57]:
trans_df[0:50]

,Transaction,Post_date,Trans_date
0,Feb 4 Feb 4 CREDIT-CASH BACK REWARD - $456.56,Feb 4,Feb 4
1,Feb 11 Feb 12 VFS Services USA Inc.101-2345678...,Feb 11,Feb 12
2,Feb 12 Feb 14 WAL-MART #1879BARSTOWCA - $17.64,Feb 12,Feb 14
3,Feb 13 Feb 14 CAPITAL ONE AUTOPAY PYMTAuthDate...,Feb 13,Feb 14
4,Jan 18 Jan 20 RIVER ROAD X-PRESS CAR WACORONAC...,Jan 18,Jan 20
5,Jan 18 Jan 20 NEW SPICE WORLDCORONACA $9.05,Jan 18,Jan 20
6,Jan 20 Jan 21 PEARSON DENTAL &MED SUPP818-362-...,Jan 20,Jan 21
7,Jan 20 Jan 22 NEW SPICE WORLDCORONACA $35.95,Jan 20,Jan 22
8,Jan 22 Jan 24 INTUIT *QuickBooks OnlineCL.INTU...,Jan 22,Jan 24
9,Jan 22 Jan 24 CAFE RIO 0110 RANCHO CUCRANCHO C...,Jan 22,Jan 24


In [58]:
trans_df = trans_df.fillna(value='')

In [59]:
# Create Amount columns     
trans_df['Amount'] = trans_df.Transaction.str.extract(pat=num_pat,expand=False).str.replace(pat=r'[$\s]',repl='',regex=True)

# Create the Description columns
# len upper will prase total len of the date which is not included the space
trans_df['len_upper']=trans_df[['Trans_date','Post_date']].apply(lambda x: len(x['Trans_date'])+len(x['Post_date']), axis=1)

trans_df['len_lower'] = trans_df.Transaction.str.extract(num_pat,expand=False).apply(lambda x: len(x))

trans_df['Description'] = trans_df.apply(lambda x: get_description(trans=x['Transaction'],
                                                                   len_upper=x['len_upper'],
                                                                   len_lower=x['len_lower']),axis=1)

In [60]:
trans_df.head(20)

,Transaction,Post_date,Trans_date,Amount,len_upper,len_lower,Description
0,Feb 4 Feb 4 CREDIT-CASH BACK REWARD - $456.56,Feb 4,Feb 4,-456.56,10,9,CREDIT-CASH BACK REWARD
1,Feb 11 Feb 12 VFS Services USA Inc.101-2345678...,Feb 11,Feb 12,-178.00,12,9,VFS Services USA Inc.101-23456789DC
2,Feb 12 Feb 14 WAL-MART #1879BARSTOWCA - $17.64,Feb 12,Feb 14,-17.64,12,8,WAL-MART #1879BARSTOWCA
3,Feb 13 Feb 14 CAPITAL ONE AUTOPAY PYMTAuthDate...,Feb 13,Feb 14,"-7,549.68",12,11,CAPITAL ONE AUTOPAY PYMTAuthDate 13-Feb
4,Jan 18 Jan 20 RIVER ROAD X-PRESS CAR WACORONAC...,Jan 18,Jan 20,14.00,12,7,RIVER ROAD X-PRESS CAR WACORONACA
5,Jan 18 Jan 20 NEW SPICE WORLDCORONACA $9.05,Jan 18,Jan 20,9.05,12,6,NEW SPICE WORLDCORONACA
6,Jan 20 Jan 21 PEARSON DENTAL &MED SUPP818-362-...,Jan 20,Jan 21,663.09,12,8,PEARSON DENTAL &MED SUPP818-362-2600CA
7,Jan 20 Jan 22 NEW SPICE WORLDCORONACA $35.95,Jan 20,Jan 22,35.95,12,7,NEW SPICE WORLDCORONACA
8,Jan 22 Jan 24 INTUIT *QuickBooks OnlineCL.INTU...,Jan 22,Jan 24,25.00,12,7,INTUIT *QuickBooks OnlineCL.INTUIT.COMCA
9,Jan 22 Jan 24 CAFE RIO 0110 RANCHO CUCRANCHO C...,Jan 22,Jan 24,11.43,12,7,CAFE RIO 0110 RANCHO CUCRANCHO CUCAMOCA


In [76]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Transaction  94 non-null     object
 1   Post_date    94 non-null     object
 2   Trans_date   94 non-null     object
 3   Amount       94 non-null     object
 4   len_upper    94 non-null     int64 
 5   len_lower    94 non-null     int64 
 6   Description  94 non-null     object
dtypes: int64(2), object(5)
memory usage: 5.3+ KB


In [27]:
trans_df[['Transaction','Trans_date','Post_date','Description','Amount']].to_excel(f'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/{base_name}.xlsx',index=False)

In [25]:
dict_info[base_name]["Dataframe"] = trans_df

In [26]:
dict_info[base_name]["Dataframe"]

,Transaction,Post_date,Trans_date,Amount,len_upper,len_lower,Description
0,"10/01/2022 Beginning Balance $36,615.78",10/01/2022,,"36,615.78",10,11,Beginning Balance
1,"10/31/2022 Ending Balance $36,624.74",10/31/2022,,"36,624.74",10,11,Ending Balance
2,"10/01/2022 Beginning Balance $36,615.78",10/01/2022,,"36,615.78",10,11,Beginning Balance
3,"10/03/2022 MOBILE DEPOSIT $1,225.00",10/03/2022,,"1,225.00",10,10,MOBILE DEPOSIT
4,10/03/2022 BANKCARD-1392 BTOT DEP 512178340158...,10/03/2022,,"2,319.10",10,10,BANKCARD-1392 BTOT DEP 512178340158674
...,...,...,...,...,...,...,...
84,"10/31/2022 #0181 $4,290.00",10/31/2022,,"4,290.00",10,10,#0181
85,10/06/2022 10/11/2022 $296.32,10/06/2022,,296.32,10,8,10/11/2022
86,10/11/2022 #0261 $296.32,10/11/2022,,296.32,10,8,#0261
87,10/17/2022 10/25/2022 $123.10,10/17/2022,,123.10,10,8,10/25/2022


# Get Check NO

In [31]:
pdf_file

Document('C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/Checking e-Statement - 10_31_2022 - IRIFF ULEP DDS INC.pdf')

In [99]:
bank_of_hope = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/2022.09.30.pdf' 

In [163]:
US_bank_CHK = r'C:\Users\trilnd\Desktop\QBO\File PDF\Bank Statement/US Bank CHK.pdf'
Chase_check = r'C:\Users\trilnd\Desktop\QBO\File PDF\Bank Statement/Chase_Check.pdf'

In [164]:
pdf_file = fitz.open(Chase_check)
pdf_file

Document('C:\Users\trilnd\Desktop\QBO\File PDF\Bank Statement/Chase_Check.pdf')

In [170]:
number_page= pdf_file.page_count

In [112]:
#filter_list

In [169]:
pdf_file[2]

page 2 of C:\Users\trilnd\Desktop\QBO\File PDF\Bank Statement/Chase_Check.pdf

In [171]:
 # Get Page and Coodinator contain "Checks" word
String = 'Checks'
Pagelist = []
for i in range(0, number_page):
    PageObj = pdf_file[i]
    Text = PageObj.get_text('text')
    ReSearch = re.search(String, Text, flags=re.IGNORECASE)
    if ReSearch != None:
        # print(re.findall(String,Text))
        info = (i, ReSearch.span()[0])
        Pagelist.append(info)

In [172]:
Pagelist

[(0, 852), (1, 169), (2, 1500)]

In [173]:
checkno_text_page = ''
for i in Pagelist:
    checkno_text_page += (pdf_file[i[0]].get_text('text')[i[1]:] + '\n')

filter_list = [line.strip() for line in checkno_text_page.split('\n') if re.search(r'[a-zA-Z+]',line) is None]

In [175]:
#print(checkno_text_page)

In [177]:
#filter_list

In [156]:
pat_check_no = r'\b\d+\*?\b'
pat_date =  r'((\d{1,2}/\d{1,2})(/\d{4})?)'
pat_num = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'

In [157]:
check_no_list = []
for i in range(len(filter_list)):
    if i <len(filter_list)-1:
        line = filter_list[i]
        next_line = filter_list[i+1]
        if re.search(pat_check_no,line) and re.search(pat_date,line):
            if re.search(pat_num,next_line):
                check_no_line = line+' '+next_line
                check_no_list.append(check_no_line.split())

In [116]:
#check_no_list

In [158]:
check_no_df = pd.DataFrame(check_no_list,columns=['Check_no','Date','Amount'])
check_no_df['Amount'] = check_no_df['Amount'].str.replace('$','',regex=False)

In [159]:
check_no_df

,Check_no,Date,Amount


# TEST

## Bank of Hope

In [118]:
dir_ = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/2022.09.30.pdf' 
excel_path = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/' 

In [136]:
hope_dict = hope_bank_to_csv(pdf_path=dir_,excel_path=excel_path)

2022.09.30: Connect file successful
File 2022.09.30.pdf: 252 transactions




In [130]:
hope_dict['2022.09.30']['Dataframe']

,Transaction,Trans_date,Post_date,Amount,Description
0,"09/01/2022 Beginning Balance $132,060.81",,09/01/2022,"132,060.81",Beginning Balance
1,"09/30/2022 Ending Balance $109,998.68",,09/30/2022,"109,998.68",Ending Balance
2,"09/12/2022 DEPOSIT $12,567.00",,09/12/2022,"12,567.00",DEPOSIT
3,09/01/2022 BANKCARD SERVICE CR CD DEP 96579790...,,09/01/2022,61.98,BANKCARD SERVICE CR CD DEP 965797900778148
4,09/01/2022 BANKCARD SERVICE CR CD DEP 96579790...,,09/01/2022,"2,288.22",BANKCARD SERVICE CR CD DEP 965797900778148
...,...,...,...,...,...
247,"09/27/2022 #8444 $5,400.79",,09/27/2022,"5,400.79",#8444
248,09/29/2022 09/29/2022 $588.26,,09/29/2022,588.26,09/29/2022
249,09/29/2022 #8445 $588.26,,09/29/2022,588.26,#8445
250,09/30/2022 09/30/2022 $425.55,,09/30/2022,425.55,09/30/2022


In [132]:
hope_dict['2022.09.30'].keys()

dict_keys(['Status', 'Transaction', 'Dataframe', 'DF_Check_No'])

In [134]:
hope_dict['2022.09.30']['DF_Check_No']

,Check_no,Date,Amount
0,8356,09/12/2022,"5,000.00"
1,8371*,09/06/2022,"1,080.23"
2,8378*,09/01/2022,"8,786.10"
3,8379,09/01/2022,"2,500.00"
4,8380,09/01/2022,6.20
...,...,...,...
69,09/08/2022,"$125,918.85",None
70,09/09/2022,"$112,559.34",None
71,09/12/2022,"$115,803.68",None
72,09/13/2022,"$123,016.96",None


## Bank Panacea

In [42]:
dir_ = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/Checking e-Statement - 10_31_2022 - IRIFF ULEP DDS INC.pdf' 
excel_path = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/' 
base_name = os.path.basename(dir_)[0:-4]

In [43]:
panacea_dict = hope_bank_to_csv(pdf_path=dir_,excel_path=excel_path)

Checking e-Statement - 10_31_2022 - IRIFF ULEP DDS INC: Connect file successful
File Checking e-Statement - 10_31_2022 - IRIFF ULEP DDS INC.pdf: 89 transactions




In [44]:
panacea_dict[base_name]['Dataframe']

,Transaction,Trans_date,Post_date,Amount,Description
0,"10/01/2022 Beginning Balance $36,615.78",,10/01/2022,"36,615.78",Beginning Balance
1,"10/31/2022 Ending Balance $36,624.74",,10/31/2022,"36,624.74",Ending Balance
2,"10/01/2022 Beginning Balance $36,615.78",,10/01/2022,"36,615.78",Beginning Balance
3,"10/03/2022 MOBILE DEPOSIT $1,225.00",,10/03/2022,"1,225.00",MOBILE DEPOSIT
4,10/03/2022 BANKCARD-1392 BTOT DEP 512178340158...,,10/03/2022,"2,319.10",BANKCARD-1392 BTOT DEP 512178340158674
...,...,...,...,...,...
84,"10/31/2022 #0181 $4,290.00",,10/31/2022,"4,290.00",#0181
85,10/06/2022 10/11/2022 $296.32,,10/06/2022,296.32,10/11/2022
86,10/11/2022 #0261 $296.32,,10/11/2022,296.32,#0261
87,10/17/2022 10/25/2022 $123.10,,10/17/2022,123.10,10/25/2022


## Bank Chase

In [38]:
dir_ = r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of Hope Checking 0933 2022\20221021-statements-4293-.pdf'
excel_path = r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of Hope Checking 0933 2022'
base_name = os.path.basename(dir_)[0:-4]
base_name

'20221021-statements-4293-'

In [41]:
chase_dict = hope_bank_to_csv(pdf_path=dir_,excel_path=excel_path)

20221021-statements-4293-: Connect file successful
File 20221021-statements-4293-.pdf: 31 transactions


